Màster en ciència de dades UOC - Pràctica de visualització de dades - Pau Mazcuñán Garcia 

# Anàlisi del rendiment acadèmic dels estudiants a Bangladesh

Font dataset original: [Kaggle](https://www.kaggle.com/datasets/satayjit/student-performance-bd/data)

## Objectius

L'objectiu de la pràctica que m'he proposat és crear les visualitzacions següents:

* Veure si hi ha alguna relació entre el nivell educatiu del pare i la mare i el rendiment acadèmic. A priori sembla intuïtiu pensar que com més alt és el nivell educatiu dels pares, més estabilitat econòmica tenen i per tant l'estudiant disposa de més facilitats per a l'estudi. 
* Veure si hi ha alguna relació entre el temps d'estudi dedicat i les activitats extraescolars, i alhora relacionat amb el rendiment acadèmic. És a dir, podem pensar que si participen a activitats extraescolars hi ha menys temps per a l'estudi i això es tradueix en un pitjor rendiment acadèmic.
* Veure si hi ha "correlacions" fortes de diverses variables categòriques amb el rendiment acadèmic. Per exemple l'accés a Internet, la intervenció dels pares en els estudis del seu fill o filla, el fet que els pares tinguin feina, el tipus d'escola.

Per explorar aquestes hipòtesis amb visualitzacions de dades necessitarem fer un preprocessament correcte del dataset com el que detallem a continuació:

* El dataset original mostra el rendiment acadèmic desglossat per camps (arts, ciència, llengua, etc.). Per simplificar-ho crearem un nou camp *avg_grade* fent la mitjana entre les variables dels camps específics.
* Unificar les variables binàries *mother_job* i *father_job* amb una nova variable *parents_job* amb tres categories, en funció de si no tenen feina cap dels dos, només un o bé els dos.
* Eliminar les variables innecessàries.

## Preprocessament i neteja de les dades

Carreguem el dataset i vegem algunes dades

In [1]:
import pandas as pd

In [2]:
raw_data = pd.read_csv("../data/bd_students_per_v2.csv")
raw_data.head()

,id,full_name,age,gender,location,family_size,mother_education,father_education,mother_job,father_job,...,tutoring,school_type,attendance,extra_curricular_activities,english,math,science,social_science,art_culture,stu_group
0,2,Avi Biswas,16,Male,Urban,6,SSC,HSC,No,No,...,Yes,Private,95,Yes,95,98,92,94,98,Science
1,3,Taslima Sultana,18,Female,Rural,6,SSC,HSC,No,Yes,...,No,Semi_Govt,92,No,65,71,40,78,80,Commerce
2,4,Md Adilur Rahman,15,Male,Rural,4,SSC,SSC,Yes,Yes,...,Yes,Govt,81,Yes,64,78,58,86,74,Commerce
3,5,Saleh Ahmed,16,Male,Rural,6,SSC,SSC,Yes,Yes,...,Yes,Private,90,Yes,84,90,85,86,88,Science
4,6,Din Islam,17,Male,Urban,5,Honors,Masters,No,Yes,...,Yes,Semi_Govt,75,Yes,54,70,45,79,76,Commerce


Comencem per crear la nova variable *avg_grade*

In [3]:
data = raw_data.copy()
data["avg_grade"] = data[["english", "math", "science", "social_science", "art_culture"]].mean(axis=1)
data[["english", "math", "science", "social_science", "art_culture", "avg_grade"]].head()

,english,math,science,social_science,art_culture,avg_grade
0,95,98,92,94,98,95.4
1,65,71,40,78,80,66.8
2,64,78,58,86,74,72.0
3,84,90,85,86,88,86.6
4,54,70,45,79,76,64.8


A continuació creem la nova variable *parents_job* que tindrà valor 0 si ni el pare ni la mare treballa, 1 si un dels dos treballa o 2 si els dos treballen.

In [4]:
data["parents_job"] = data.apply(lambda row: (row["mother_job"] == "Yes") + (row["father_job"] == "Yes"), axis=1)
data[["parents_job", "mother_job", "father_job"]].head()

,parents_job,mother_job,father_job
0,0,No,No
1,1,No,Yes
2,2,Yes,Yes
3,2,Yes,Yes
4,1,No,Yes


Fixem-nos amb les variables *Mother_education* i *Father_education* dos dels valors possibles són "Honors" i "Hons".

In [5]:
print(f"Father_education values: {list(data["father_education"].unique())}")
print(f"Mother_education values: {list(data["mother_education"].unique())}")

Father_education values: ['HSC', 'SSC', 'Masters', 'Diploma', 'Honors', 'Hons', 'Under_SSC', 'Non_Educated', nan]
Mother_education values: ['SSC', 'Honors', 'HSC', 'Diploma', 'Masters', 'Under_SSC', 'Hons', 'Non_Educated', nan]


Suposarem que "Hons" i "Honors" són equivalents i unificarem el valor.

In [6]:
data["father_education"] = data["father_education"].replace("Hons", "Honors")
data["mother_education"] = data["mother_education"].replace("Hons", "Honors")

Per acabar el preprocessament veiem totes les variables que té el nou dataframe i n'eliminem les innecessàries.

In [7]:
print(data.columns)

Index(['id', 'full_name', 'age', 'gender', 'location', 'family_size',
       'mother_education', 'father_education', 'mother_job', 'father_job',
       'guardian', 'parental_involvement', 'internet_access', 'studytime',
       'tutoring', 'school_type', 'attendance', 'extra_curricular_activities',
       'english', 'math', 'science', 'social_science', 'art_culture',
       'stu_group', 'avg_grade', 'parents_job'],
      dtype='object')


In [8]:
data_cleaned = data[["mother_education",
                     "father_education",
                     "studytime",
                     "extra_curricular_activities",
                     "parents_job",
                     "internet_access",
                     "parental_involvement",
                     "school_type",
                     "avg_grade"]]
data_cleaned.head()

,mother_education,father_education,studytime,extra_curricular_activities,parents_job,internet_access,parental_involvement,school_type,avg_grade
0,SSC,HSC,8,Yes,0,Yes,Yes,Private,95.4
1,SSC,HSC,4,No,1,No,Yes,Semi_Govt,66.8
2,SSC,SSC,5,Yes,2,Yes,Yes,Govt,72.0
3,SSC,SSC,7,Yes,2,Yes,Yes,Private,86.6
4,Honors,Masters,4,Yes,1,Yes,Yes,Semi_Govt,64.8


Finalment explorem el dataset final si hi ha valors absents o nuls.

In [9]:
data_cleaned.isna().sum()

mother_education               10
father_education                4
studytime                       0
extra_curricular_activities     0
parents_job                     0
internet_access                 0
parental_involvement            0
school_type                     0
avg_grade                       0
dtype: int64

In [10]:
(data_cleaned == "").sum()

mother_education               0
father_education               0
studytime                      0
extra_curricular_activities    0
parents_job                    0
internet_access                0
parental_involvement           0
school_type                    0
avg_grade                      0
dtype: int64

Observem que les variables *Mother_education* i *Father_education* tenen alguns valors nuls. Els eliminem perquè són pocs i els podem descartar.

In [11]:
data_cleaned = data_cleaned.dropna()

## Exportació del dataset

Ja tenim el dataset a punt, aleshores exportem el dataframe en format csv per passar-lo al Tableau, l'eina que farem servir per a crear les visualitzacions de dades esmentades. El nou dataset es troba a *data/bd_students_cleaned.csv*

In [12]:
with open("../data/bd_students_cleaned.csv", "w") as f:
    data_cleaned.to_csv(f, index=False)